# V12 (gated 82+83): SWEET

Re-runs the SWEET test on the gated (82+83) dataset to check for sinusoidal variability at the transit period and common aliases.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

sectors_used = [82, 83]
ds = sh.load_dataset()
lc = sh.stitch_pdcsap_sectors(ds, sectors_used)
depth_ppm, depth_err_ppm = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False, preset='extended')
r = session.run('V12')

print(json.dumps({
  'sectors_used': sectors_used,
  'depth_ppm': depth_ppm,
  'depth_err_ppm': depth_err_ppm,
  'status': r.status,
  'flags': list(r.flags),
  'metrics': r.metrics,
}, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "depth_err_ppm": 11.153641839855947,
  "depth_ppm": 264.57118442002604,
  "flags": [],
  "metrics": {
    "baseline_days": 50.98,
    "cadence_median_min": 2.0,
    "n_points": 35459,
    "n_transits_expected": 3,
    "snr_at_period": 6.901905743025918,
    "snr_double_period": 1.1145785781293491,
    "snr_half_period": 1.8373496539712335,
    "sweet_msg": "WARN: SWEET test finds signal at the transit period"
  },
  "sectors_used": [
    82,
    83
  ],
  "status": "ok"
}
```

</details>


In [ ]:
import json
import matplotlib.pyplot as plt

from bittr_tess_vetter.plotting import plot_sweet

step_id = '30_gated_82_83_v12_sweet'
fname = 'V12_sweet.png'

run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)
run_path = run_out_dir / fname
docs_path = (docs_out_dir / fname) if docs_out_dir is not None else None

out = {'status': r.status, 'flags': list(r.flags)}

if r.status == 'ok':
    fig, ax = plt.subplots(figsize=(9, 5))
    plot_sweet(r, ax=ax)
    ax.set_title('V12 (gated 82+83): SWEET')
    fig.tight_layout()
    fig.savefig(run_path, dpi=160, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=160, bbox_inches='tight')
    plt.show()
    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/30_gated_82_83_v12_sweet/V12_sweet.png",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/30_gated_82_83_v12_sweet/V12_sweet.png",
  "status": "ok"
}
```

</details>


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/30_gated_82_83_v12_sweet/V12_sweet.png" width="820" />


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** SWEET warning persists (`sweet_msg` indicates variability at the transit period); SNR at P is elevated (~6.9) relative to P/2 and 2P.
- **Why this is useful:** variability at the transit period can mimic or distort shallow transit signals.
- **Interpretation:** persistent warning supports the V16 “transit+sinusoid” preference and suggests careful detrending/variant selection is required.
- **Next step:** re-run V01–V05 (odd/even, secondary, duration, depth stability, shape) on the gated dataset to see if classic EB/shape checks improve under gating.

</details>
